<a href="https://colab.research.google.com/github/saraahmedi20002-source/git-test/blob/main/our_approach_unstructured.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import csv

from google.colab import files
uploaded = files.upload()
from google.colab import files
uploaded = files.upload()
from google.colab import files
uploaded = files.upload()

# Lire le fichier JSON ligne par ligne
df = pd.read_json("scholar.json", lines=True)
# Sauvegarder en CSV
df.to_csv("scholar.csv", index=False, encoding="utf-8")
# print("scholar.csv créé avec succès.")

df = pd.read_json("dblp.json", lines=True)
# Sauvegarder en CSV
df.to_csv("dblp.csv", index=False, encoding="utf-8")
# print("dblp.csv créé avec succès.")

#mal9itch hada fichier dok rayh nbdlo b hdu line mnb3d nas9si chikha
#df = pd.read_json("scholar-dblp.json", lines=True)
df = pd.read_json("scholardblp.json", lines=True)


# Sauvegarder en CSV
df.to_csv("reference.csv", index=False, encoding="utf-8")
# print("reference.csv créé avec succès.")

!pip install jellyfish
import pandas as pd

import re
import json
from collections import defaultdict
from itertools import combinations
from jellyfish import jaro_similarity  # pip install jellyfish

dblp_df = pd.read_csv(r'dblp.csv', encoding='latin1', dtype={'id': str})
dblp_df['source'] = 'dblp'
dblp_df['realProfileID'] = dblp_df['realProfileID'].apply(lambda x: f'dblp{x}')

scholar_df = pd.read_csv(r'scholar.csv', encoding='latin1', dtype={'id': str})
scholar_df['source'] = 'scholar'
scholar_df['realProfileID'] = scholar_df['realProfileID'].apply(lambda x: f'scholar{x}')
import time
start_time = time.time()

# Fusionner les deux datasets
combined_df = pd.concat([scholar_df, dblp_df], ignore_index=True)
combined_df.to_csv("combined_dataset.csv", index=False)


# === 2. Construction de l’index inversé sur les titres ===

# Liste simple de stop words à exclure
stop_words = {
    "the", "and", "or", "for", "of", "in", "on", "a", "an", "to", "is", "with", "by", "at", "from", "as", "that"
}

def tokenize(text):
    if pd.isna(text):
        return []
    raw_tokens = re.findall(r'\b\w+\b', str(text).lower())
    # Filtrer tokens : enlever stop words et tokens d'une seule lettre
    clean_tokens = [token for token in raw_tokens if token not in stop_words and len(token) > 1]
    return clean_tokens

def build_inverted_index_from_fields(df, fields, id_col='realProfileID'):
    index = defaultdict(set)

    for _, row in df.iterrows():
        record_id = row[id_col]
        combined_text = ' '.join(str(row[field]) for field in fields if field in row and not pd.isna(row[field]))
        tokens = tokenize(combined_text)
        for token in tokens:
            index[token].add(record_id)

    return {token: list(ids) for token, ids in index.items()}

#Exemple d'utilisation :
fields_to_use = ['title', 'venue','authors','year']  # Choisis les colonnes que tu veux combiner
text_index = build_inverted_index_from_fields(combined_df, fields_to_use)

#with open("text_index.json", "w", encoding="utf-8") as f:
#    json.dump(text_index, f, ensure_ascii=False, indent=2)
# === 3. Extraire les groupes fréquents ===

filtered_text_index = {}
refused_text_index = {}
for token, ids in text_index.items():
    if 2 <= len(ids) <= 60:
        filtered_text_index[token] = ids
    else: refused_text_index[token]=ids

# Sauvegarder filtered_title_index dans un fichier JSON
#with open("filtered_text_index.json", "w", encoding="utf-8") as f:
#    json.dump(filtered_text_index, f, ensure_ascii=False, indent=2)

all_pairs=set()
candidate_pairs=set()
candidate_pair_counts = defaultdict(int)

def from_different_sources(id1, id2):
    return (id1.startswith("dblp") and id2.startswith("scholar")) or \
           (id1.startswith("scholar") and id2.startswith("dblp"))

for token, id_list in filtered_text_index.items():
    for pair in combinations(sorted(id_list), 2):
        id1, id2 = pair
        if from_different_sources(id1, id2):
            candidate_pair_counts[pair] += 1
            candidate_pairs.add(pair)

with open("candidate_pairs_from_text.csv", "w", newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["id1", "id2"])
    writer.writerows(candidate_pairs)

all_ids = set()

for index in [filtered_text_index, refused_text_index]:
    for id_list in index.values():
        all_ids.update(id_list)

# Créer un mapping id → index
id_to_index = {id: idx for idx, id in enumerate(sorted(all_ids))}

# Incrémenter uniquement les paires qui sont déjà dans candidate_pair_counts
for token, ids in refused_text_index.items():
   # ids = [id for id in ids if id in id_to_index]  # sécurité : s'assurer que l'id est indexé
    for id1, id2 in combinations(sorted(ids), 2):
        if (id1, id2) in candidate_pair_counts :
         #if from_different_sources(id1, id2):
           candidate_pair_counts[(id1, id2)] += 1

# Charger les deux fichiers
combined_df = pd.read_csv("combined_dataset.csv")
#pairs_df = pd.read_csv("candidate_pairs_from_text.csv")

# Charger les candidats existants
candidate_df = pd.read_csv("candidate_pairs_from_text.csv")
candidate_pairs = set(tuple(sorted([row['id1'], row['id2']])) for _, row in candidate_df.iterrows())

# Charger les index filtrés
#with open('filtered_text_index.json', 'r', encoding='utf-8') as f:
 #   filtered_index_data = json.load(f)

all_token_groups = []

# Pondération avec contrainte "sources différentes"
pair_weights = defaultdict(int)


# Calculer la longueur totale du texte pour chaque profil

stop_words = {
    "the", "and", "or", "for", "of", "in", "on", "a", "an", "to", "is", "with",
    "by", "at", "from", "as", "that", "this", "these", "those", "it", "be", "are", "was"
}

# Fonction de nettoyage et tokenisation
def tokenize_and_clean(text):
    # Extraire les mots alphanumériques, passer en minuscules
    tokens = re.findall(r'\b\w+\b', str(text).lower())
    # Supprimer les stop words et tokens d'une seule lettre
    return [
        token for token in tokens
        if token not in stop_words and len(token) > 1
    ]

# Dictionnaire final : ID → nombre de mots (après nettoyage)
id_to_text_length = {}

# Lecture du fichier CSV
with open("combined_dataset.csv", newline='', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        profile_id = row["realProfileID"]
        total_words = 0

        # Parcours de toutes les colonnes sauf l'ID
        for key, value in row.items():
            if key == "realProfileID":
                continue
            if value:
                tokens = tokenize_and_clean(value)
                total_words += len(tokens)

        id_to_text_length[profile_id] = total_words

# Étape 2 : Écrire dans weighted_candidate_pairs.csv avec les longueurs totales
with open("weighted_candidate_pairs.csv", "w", newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["id1", "id2", "weight", "text_len_id1", "text_len_id2"])

    for (id1, id2), weight in candidate_pair_counts.items():
        len1 = id_to_text_length.get(id1, 0)
        len2 = id_to_text_length.get(id2, 0)

        writer.writerow([id1, id2, weight, len1, len2])

# --- Filtrage final avec seuil ---
df = pd.read_csv("weighted_candidate_pairs.csv")

df_filtered = df[df['weight'] > 6].copy()

df_filtered.to_csv("filtered_weighted_pairs.csv", index=False)
print(f"filtered_weighted_pairs.csv    {len(df_filtered)}")


"""# **Claculer la similarité **"""

final_results = []

# === Fin du chronométrage ===

df = pd.read_csv("filtered_weighted_pairs.csv", header=None)
df = df.drop(columns=[2])
# Permuter les deux premières colonnes
df[[0, 1]] = df[[1, 0]]

# Enregistrer le nouveau fichier
df.to_csv("filtered_weighted_pairs2.csv", index=False, header=False)
final_df = pd.read_csv("filtered_weighted_pairs2.csv")
#final_df = df
def extract_original_ids(row):
    if row['id1'].startswith('dblp') and row['id2'].startswith('scholar'):
        id_dblp = row['id1'].replace('dblp', '')
        id_acm = row['id2'].replace('scholar', '')
    elif row['id1'].startswith('scholar') and row['id2'].startswith('dblp'):
        id_dblp = row['id2'].replace('dblp', '')
        id_acm = row['id1'].replace('scholar', '')
    else:
        return None
    return pd.Series([id_dblp, id_acm])

final_df[['id1', 'id2']] = final_df.apply(extract_original_ids, axis=1)
final_pairs = final_df.dropna(subset=['id1', 'id2'])[['id1', 'id2']]


final_pairs.to_csv("filtered_weighted_pairs3.csv", index=False)

print(f"filtered_weighted_pairs3.csv {len(final_pairs)} ")


resultats = pd.read_csv(r'filtered_weighted_pairs3.csv')
#resultats=final_pairs
reference = pd.read_csv(r'reference.csv')


set_resultats = set([tuple(x) for x in resultats[['id1', 'id2']].values])
set_reference = set([tuple(x) for x in reference[['id1', 'id2']].values])


true_positives = set_resultats.intersection(set_reference)
false_positives = set_resultats - set_reference
false_negatives = set_reference - set_resultats

# Sauvegarde des faux négatifs dans un CSV
#df_fn = pd.DataFrame(list(false_negatives), columns=['iddblp', 'idscholar'])
#df_fn.to_csv("false_negatives.csv", index=False)

#print("Fichier 'false_negatives.csv' créé avec", len(df_fn), "lignes.")
tp = len(true_positives)
fp = len(false_positives)
fn = len(false_negatives)

precision = tp / (tp + fp) if (tp + fp) != 0 else 0
recall = tp / (tp + fn) if (tp + fn) != 0 else 0
f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) != 0 else 0


print(f"True Positives (TP): {tp}")
print(f" False Positives (FP): {fp}")
print(f" False Negatives (FN): {fn}")
print(f"\n Precision: {precision:.2f}")
print(f" Recall: {recall:.2f}")
print(f" F1 Score: {f1_score:.2f}")

end_time = time.time()
execution_time = end_time-start_time
print("Temps d'exécution (s) :", execution_time)


Saving scholarr.json to scholarr (1).json


Saving dblp.json to dblp (2).json


Saving scholardblp.json to scholardblp (1).json
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.5/360.5 kB 24.0 MB/s eta 0:00:00
filtered_weighted_pairs.csv    2727
filtered_weighted_pairs3.csv 2727 
True Positives (TP): 1977
 False Positives (FP): 750
 False Negatives (FN): 331

 Precision: 0.72
 Recall: 0.86
 F1 Score: 0.79
Temps d'exécution (s) : 97.01433682441711
